In [1]:
from util.sparkhandler import SparkHandler
from util.dataproctools import get_extracted_wet

We use the spark handler class so we can use a unified spark session across our functions. This essentially just helps us initialize the spark session and stores the session and context within itself

In [ ]:
handler = SparkHandler(available_cores=4) # look at the parameters for SparkHandler to increase max memory
ss = handler.get_spark_session()
sc = handler.get_spark_context()

Here we can load in our str data. Do note, `get_extracted_wet` doesn't return the sample size exactly, but an approximate amount based on the wet we use

In [3]:
loaded_str_data = get_extracted_wet(spark_context=sc, approx_sample_size=100000, wet_paths_file="../data/wet.paths") #load_rdd(spark_context=sc, path_to_load="../saved_intermediates/rawStrRDD")

Here's what the start of a WET file looks like

And this is what it looks like when printed in a prettier format

In [4]:
def split_sites(string):
    import re
    # The header of every item is WARC/1.0\r\nWARC-Type: conversion, but we assume that the WARC version could be different
    return re.split(r"WARC/\d.\d\r\nWARC-Type: conversion\r\n", string)[1:]

In [5]:
split_rdd = loaded_str_data.flatMap(split_sites)

In [6]:
def regex_split_to_kv(string):
    import re
    return re.split(r"Content-Length: \d+\r\n", string)

In [7]:
KV_mapped = split_rdd.map(lambda x: tuple(regex_split_to_kv(x)))

In [8]:
Ksplit = KV_mapped.map(lambda x: (x[0].split('\r\n') ,x[1]))

In [9]:
def get_warc_target_uri(split_key):
    return split_key[0].split(': ')[1]
def get_warc_target_date(split_key):
    return split_key[1].split(': ')[1]
def get_warc_record_id(split_key):
    return split_key[2].split(': ')[1]
def get_warc_languages(split_key):
    return split_key[5].split(': ')[1].split(',')
def get_tld_from_url(url):
    import re
    # search for strings that start with "." and are followed by a "/" that are at least 2 letters long and also search for also that russian tld approximation that makes no sense
    searched = re.search(string=url, pattern=r"\.((?:xn--[a-z0-9]+)|[a-z]{2,})(?=[/?#:]|$)")
    return searched[0] if searched is not None else "None"
get_tld_from_url("http://022hezi.com/tag/%E4%B9%9D%E5%8D%81%E5%85%AB")

'.com'

In [10]:
with_processed_keys = Ksplit.map(lambda x:
                                 (
                                     get_warc_record_id(x[0]),
                                     get_warc_target_uri(x[0]),
                                     get_warc_target_date(x[0]),
                                     get_warc_languages(x[0]),
                                     x[1]
                                 )
                                 )

In [24]:
ex = with_processed_keys.take(20)

In [11]:
tlds_added = with_processed_keys.map(lambda x: (x[0], x[1], x[2], x[3], get_tld_from_url(x[1]), x[4]))

In [28]:
df = ss.createDataFrame(tlds_added, schema=['warc_id', 'target_uri', 'date', 'languages', 'tld', 'raw_content'])

In [1]:
df.toPandas()

NameError: name 'df' is not defined

In [14]:
ss.stop()